In [15]:
!pip install pandas
!pip install optuna
!pip install optuna-dashboard
!pip install scikit
!pip install imbalanced-learn
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [16]:
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# print(X.describe())
# X.drop(columns = ['Age', 'Gender', 'UAlb', 'Ucr', 'UACR', 'LDLC', 'HDLC'], inplace=True)

# [Age,Gender,UAlb,Ucr,UACR,TC,TG,TCTG,LDLC,HDLC,Scr,BUN,FPG,HbA1c,Height,Weight,BMI,Duration,DR,Community_baihe,Community_chonggu,Community_huaxin,Community_jinze,Community_liantang,Community_xianghuaqiao,Community_xujin,Community_yingpu,Community_zhaoxian,Community_zhujiajiao]
#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

Using device: cuda


In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from modularModels1 import BlockMaker, modularNN, BasicModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, drop_last=True)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

Using cuda


In [18]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def augment_data_in_place(X, X_test, normalisation_method=MinMaxScaler()):
    all_numerical_columns = [
        'Age', 'Height', 'Weight', 'Duration',
        'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 
        'TCTG', 'LDLC', 'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c'
    ]

    # Find which of those columns actually exist in both X and X_test
    existing_columns = [col for col in all_numerical_columns if col in X.columns and col in X_test.columns]
    
    if not existing_columns:
        print("No matching columns found for augmentation. Normalised data only.")
        X = normalisation_method.fit_transform(X)
        X_test = normalisation_method.transform(X_test)
        return X, X_test

    # 1. Log-transform
    X_copy = X.copy()
    X_test_copy = X_test.copy()
    X_copy.loc[:, existing_columns] = X_copy.loc[:, existing_columns].apply(np.log1p)
    X_test_copy.loc[:, existing_columns] = X_test_copy.loc[:, existing_columns].apply(np.log1p)

    # 2. Add Gaussian noise to training data only
    noise = np.random.normal(0, 0.1, X_copy[existing_columns].shape)
    X_copy.loc[:, existing_columns] = X_copy.loc[:, existing_columns] + noise

    # 3. Fit scaler on train, transform both
    scaler = normalisation_method
    X_copy.loc[:, existing_columns] = scaler.fit_transform(X_copy.loc[:, existing_columns])
    X_test_copy.loc[:, existing_columns] = scaler.transform(X_test_copy.loc[:, existing_columns])

    return X_copy, X_test_copy


In [20]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None, contamination=0.05):
    """
    Generates stratified folds with specified normalization.

    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - QuantileTransformer(output_distribution='uniform')

    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []

    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        # print("Original\n", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
        X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
        if contamination is not None and contamination > 0: #? using contamination = 0.0 works
            X_train_zeros = X_train[Y_train.iloc[:, 0] == 0]
            X_train_ones = X_train[Y_train.iloc[:, 0] == 1]
            Y_train_zeros = Y_train[Y_train.iloc[:, 0] == 0]
            Y_train_ones = Y_train[Y_train.iloc[:, 0] == 1] 
            # print("Ones and zeros\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
            #only class 0s
            if X_train_zeros.isna().any().any():
                print("got NaN values in the training set")
            
            # Apply Isolation Forest to majority class only
            iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
            try:
                outliers = iso_forest.fit_predict(X_train_zeros)
            except UserWarning as e:
                print("Caught warning during IsolationForest fitting:", e)
                outliers = np.ones(len(X_train_zeros))  # If warning occurs, keep all data
            # Keep only non-outlier majority samples
            X_train_zeros = X_train_zeros[outliers == 1]
            Y_train_zeros = Y_train_zeros[outliers == 1]
            # print("After iso:\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
            
            # Combine the cleaned majority class with the untouched minority class
            X_train_cleaned = pd.concat([X_train_zeros, X_train_ones])
            Y_train_cleaned = pd.concat([Y_train_zeros, Y_train_ones])
        #? data augmentation on leftover data
        X_train_scaled, X_test_scaled = augment_data_in_place(X_train_cleaned, X_test, normalisation_method=normalisation_method)
        
        # Handle oversampling if needed
        #! use X_train_scaled and Y_train_cleaned for oversampling becasue y_train_cleaned no changes after augmentation
        if oversampler:
            X_train_scaled, Y_train_cleaned = oversampler.fit_resample(X_train_scaled, Y_train_cleaned)

        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_cleaned.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')]
        if community_cols:
            X_train_scaled[community_cols] = X_train_scaled[community_cols].apply(
                lambda row: pd.Series(np.eye(len(row))[row.argmax()]), axis=1
            ).set_axis(community_cols, axis=1)
        # print(X_train_scaled[community_cols].describe())

        # Ensure 'Gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)

        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train_cleaned, Y_test))

        print(f"Fold: {fold+1}, Train: {X_train_scaled.shape}, Test: {X_test_scaled.shape}")

    return kFolds_list

In [21]:
# oversampler = None
# contamination = 0.05
# normalisation_method = QuantileTransformer()
# kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
#                          normalisation_method = normalisation_method, 
#                          n_splits=5, 
#                          oversampler = oversampler, random_state=42, contamination=contamination)

In [37]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device, threshold = 0.5):
    if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0

    
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            outputs = model(inputs) #? Forward pass through the model
            loss = criterion(outputs, labels) #? Calculate loss
            loss.backward() #? Backpropagation
            running_loss += loss.item()
            optimiser.step() #? Update weights
            if scheduler:
                if isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
                    scheduler.step()
        # if scheduler: 
        #     if not isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
        #         scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
            scheduler.step(avg_val_loss) if scheduler and not isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR) else None
            # print(scheduler.get_last_lr())
            # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
            # print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.4f}".ljust(12), f"Val Loss: {avg_val_loss:.4f}",end="\r")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        elif avg_val_loss*0.95 <= best_val_loss:
                wait = 0
        else:
            wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.4f}".ljust(12), f"best_val_loss:{best_val_loss:.4f}".ljust(12), f"Val Loss: {avg_val_loss:.4f}", f"Scheduler lr: {scheduler.get_last_lr()}",end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                # predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                predictions = (torch.sigmoid(outputs) >= threshold).float().cpu().numpy()

                
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc


In [38]:
import torch
import torch.nn as nn

class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=None, activation=nn.ReLU):
        super().__init__()
        layers = [
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation()
        ]
        if dropout and dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)


class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout= None, hidden_dim2= None):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout)
        self.block2 = FeedForwardBlock(hidden_dim, output_dim, dropout)
        # self.block3 = FeedForwardBlock(hidden_dim2, output_dim, dropout = None)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        # x = self.block3(x)
        x = self.output_layer(x)  # Final linear layer
        return x
    
    def last_layer(self):
        return self.output_layer
test_model = MyModel(8,64,16,0)
print(test_model)


MyModel(
  (block1): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (block2): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (output_layer): Linear(in_features=16, out_features=1, bias=True)
)


In [41]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 10000
    random_state = 42
    oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=random_state)
    if isinstance(oversampler, ADASYN):
        n_neighbours = trial.suggest_int("n_neighbours", 2, 10)
        oversampler = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbours, random_state=random_state)
    
    normalisation_method = trial.suggest_categorical("normalisation_method", [
        "MinMaxScaler",
        # "MaxAbsScaler",
    #     "StandardScaler",
    #     # "PowerTransformer",
        # "QuantileTransformer",
    ])
    if normalisation_method:
        if normalisation_method == "MinMaxScaler":
            normalisation_method = MinMaxScaler()

        elif normalisation_method == "StandardScaler":
            normalisation_method = StandardScaler()
        elif normalisation_method == "RobustScaler":
            normalisation_method = RobustScaler()
        elif normalisation_method == "PowerTransformer":
            normalisation_method = PowerTransformer()
        elif normalisation_method == "QuantileTransformer":
            normalisation_method = QuantileTransformer(output_distribution='uniform')
        else:
            normalisation_method = MinMaxScaler()
    contamination = trial.suggest_float("contamination", 0.01, 0.2)
    kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42, contamination=contamination)
                        
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 16, 128)
    # hidden_dim2 = trial.suggest_int("hidden_dim2", 20, 100)
    hidden_dim2 = None
    output_dim = trial.suggest_int("output_dim", 2, hidden_dim)
    
    dropout = trial.suggest_float("dropout", 0.00, 0.3)
    threshold = trial.suggest_float("threshold", 0.1, 0.9)
    # dropout = None
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 5e-3, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    
    # Loss function hyperparameters
    # criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss", "FocalLoss"]) 
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss"]) 
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 10)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight= None
        alpha = trial.suggest_float("alpha", 1, 2)
        gamma = trial.suggest_float("gamma", 0.25, 2)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device)
        # Calculate steps_per_epoch from the current fold's train_loader
        train_loader_len = len(train_loader)
        
        # Instantiate and initialize the model
        # model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, dropout=dropout)
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, output_dim=hidden_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma).to(device)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=weight_decay)

        
        # Initialize scheduler
        # scheduler = torch.optim.lr_scheduler.OneCycleLR(
        #     optimiser,
        #     max_lr=max_lr,
        #     steps_per_epoch=train_loader_len,
        #     epochs=epochs,
        #     anneal_strategy='linear'
        # )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min', patience=7, factor=0.5, min_lr=1e-6)
        print(f"Fold {fold}:")
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=40, device=device, threshold = 0.5
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score


In [ ]:
import threading
import optuna
from optuna_dashboard import run_server

def start_dashboard():
    run_server(storage)

storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# Start dashboard in a separate thread
dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=30)

# After optimization, print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 14:27:58,246] A new study created in memory with name: Basic
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Using device: cuda
Fold: 1, Train: (7288, 28), Test: (1149, 28)
Fold: 2, Train: (7246, 28), Test: (1149, 28)
Fold: 3, Train: (7240, 28), Test: (1148, 28)
Fold: 4, Train: (7208, 28), Test: (1148, 28)
Fold: 5, Train: (7252, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 57, best val loss: 0.3895Loss: 0.6803 Scheduler lr: [4.437408082682723e-05]]
Accuracy: 0.6858, precision: 0.2005, recall: 0.7069, f1: 0.3124, auc: 0.6952
Fold 2:
Early stopping triggered at epoch 66, best val loss: 0.3380Loss: 0.6635 Scheduler lr: [4.437408082682723e-05]]
Accuracy: 0.6928, precision: 0.2015, recall: 0.6897, f1: 0.3119, auc: 0.6914
Fold 3:
Early stopping triggered at epoch 68, best val loss: 0.3718Loss: 0.6095 Scheduler lr: [2.2187040413413616e-05]
Accuracy: 0.7186, precision: 0.1796, recall: 0.5000, f1: 0.2642, auc: 0.6216
Fold 4:
Early stopping triggered at epoch 68, best val loss: 0.3798Loss: 0.6370 Scheduler lr: [4.437408082682723e-05]]
Accuracy: 0.6951, precision: 0.1750, recall: 0.54

[I 2025-04-15 14:29:28,331] Trial 0 finished with value: 0.28834272006635314 and parameters: {'n_neighbours': 4, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.13248425123950647, 'hidden_dim': 83, 'output_dim': 58, 'dropout': 0.2866079398541896, 'threshold': 0.16035256862318034, 'initial_lr': 0.0007099852932292357, 'max_lr': 0.005, 'weight_decay': 0.005329682180071191, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 1}. Best is trial 0 with value: 0.28834272006635314.


Early stopping triggered at epoch 58, best val loss: 0.3370
Accuracy: 0.6821, precision: 0.1864, recall: 0.6379, f1: 0.2885, auc: 0.6625
Using device: cuda
Fold: 1, Train: (6855, 28), Test: (1149, 28)
Fold: 2, Train: (6859, 28), Test: (1149, 28)
Fold: 3, Train: (6833, 28), Test: (1148, 28)
Fold: 4, Train: (6851, 28), Test: (1148, 28)
Fold: 5, Train: (6878, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 63, best val loss: 0.7039Loss: 0.8785 Scheduler lr: [1.1735255237958168e-05]
Accuracy: 0.6214, precision: 0.1725, recall: 0.7241, f1: 0.2786, auc: 0.6670
Fold 2:
Early stopping triggered at epoch 64, best val loss: 0.6781Loss: 0.9603 Scheduler lr: [4.694102095183267e-05]
Accuracy: 0.5997, precision: 0.1705, recall: 0.7672, f1: 0.2790, auc: 0.6740
Fold 3:
Early stopping triggered at epoch 66, best val loss: 0.6891Loss: 0.8463 Scheduler lr: [4.694102095183267e-05]
Accuracy: 0.6463, precision: 0.1792, recall: 0.6983, f1: 0.2852, auc: 0.6694
Fold 4:
Early stopping triggered 

[I 2025-04-15 14:30:48,234] Trial 1 finished with value: 0.27644722764565766 and parameters: {'n_neighbours': 10, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.17031717395020485, 'hidden_dim': 27, 'output_dim': 17, 'dropout': 0.01836115753896502, 'threshold': 0.3672740014381496, 'initial_lr': 0.0007510563352293228, 'max_lr': 0.005, 'weight_decay': 2.5855293605803765e-05, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 2}. Best is trial 0 with value: 0.28834272006635314.


Early stopping triggered at epoch 57, best val loss: 0.7270
Accuracy: 0.6124, precision: 0.1755, recall: 0.7672, f1: 0.2857, auc: 0.6811
Using device: cuda
Fold: 1, Train: (7519, 28), Test: (1149, 28)
Fold: 2, Train: (7459, 28), Test: (1149, 28)
Fold: 3, Train: (7476, 28), Test: (1148, 28)
Fold: 4, Train: (7789, 28), Test: (1148, 28)
Fold: 5, Train: (7508, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 5694, best val loss: 0.7874ss: 0.8816 Scheduler lr: [1e-06]704397904931e-06]
Accuracy: 0.6362, precision: 0.1841, recall: 0.7586, f1: 0.2963, auc: 0.6905
Fold 2:
Early stopping triggered at epoch 41, best val loss: 0.6812Loss: 0.8869 Scheduler lr: [9.001363518323945e-06]]
Accuracy: 0.5222, precision: 0.1514, recall: 0.8103, f1: 0.2551, auc: 0.6501
Fold 3:
Early stopping triggered at epoch 8699, best val loss: 0.7491ss: 0.8218 Scheduler lr: [1e-06]704397904931e-06]
Accuracy: 0.6298, precision: 0.1634, recall: 0.6466, f1: 0.2609, auc: 0.6372
Fold 4:
Early stopping triggere

[I 2025-04-15 16:26:04,565] Trial 2 finished with value: 0.2688649605015465 and parameters: {'n_neighbours': 2, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.07310606210035579, 'hidden_dim': 117, 'output_dim': 66, 'dropout': 0.14136576518190958, 'threshold': 0.36816401687849765, 'initial_lr': 0.00014402181629318312, 'max_lr': 0.005, 'weight_decay': 4.577095377910826e-05, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 2}. Best is trial 0 with value: 0.28834272006635314.


Early stopping triggered at epoch 3663, best val loss: 0.7974
Accuracy: 0.6098, precision: 0.1667, recall: 0.7155, f1: 0.2704, auc: 0.6567
Using device: cuda


In [ ]:
# import threading
# import optuna
# from optuna_dashboard import run_server
# # !fuser -k 8080/tcp

# # Define your persistent storage
# storage = "sqlite:///jya.db"

# # Create or load your study
# study_name = "jya"
# try:
#     study = optuna.load_study(study_name=study_name, storage=storage)
# except KeyError:
#     study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage)

# # Start Optuna Dashboard in a separate thread
# dashboard_thread = threading.Thread(target=lambda: run_server(storage), daemon=True)
# dashboard_thread.start()

# # Run optimization
# study.optimize(maximise_combined_score, n_trials=2000)

# # Print results
# print("Best trial:")
# trial = study.best_trial
# print(f"  Combined score: {trial.value}")
# print("  Best hyperparameters:")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")
